In [35]:
#https://zhuanlan.zhihu.com/p/59772104
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader
import torch.utils.data as data

train_x = [torch.Tensor([[1], [1], [1], [1], [1], [1], [1]]),
           torch.Tensor([[2], [2], [2], [2], [2], [2]]),
           torch.Tensor([[3], [3], [3], [3], [3]]),
           torch.Tensor([[4], [4], [4], [4]]),
           torch.Tensor([[5], [5], [5]]),
           torch.Tensor([[6], [6]]),
           torch.Tensor([[7]])
           ]
pad_x = rnn_utils.pad_sequence(train_x, batch_first=True)
print(train_x)
print(pad_x)
class MyData(data.Dataset):
    def __init__(self, data_seq):
        self.data_seq = data_seq

    def __len__(self):
        return len(self.data_seq)

    def __getitem__(self, idx):
        return self.data_seq[idx]


if __name__=='__main__':
    #data = MyData(train_x)
    data = MyData(pad_x)
    data_loader = DataLoader(data, batch_size=2, shuffle=True)
    batch_x = iter(data_loader).next()
    print(batch_x)
    print('END')
    #以数据的最大长度，后两位都是00，在lstm中浪费两次计算。

[tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]), tensor([[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]]), tensor([[3.],
        [3.],
        [3.],
        [3.],
        [3.]]), tensor([[4.],
        [4.],
        [4.],
        [4.]]), tensor([[5.],
        [5.],
        [5.]]), tensor([[6.],
        [6.]]), tensor([[7.]])]
tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [0.]],

        [[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [0.],
         [0.]],

        [[4.],
         [4.],
         [4.],
         [4.],
         [0.],
         [0.],
         [0.]],

        [[5.],
         [5.],
         [5.],
         [0.],
         [0.],
         [0.],
         [0.]],

        [[6.],
         [6.],
         [0.],
         [0.]

In [37]:
#pad
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data

if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=2, shuffle=True, 
                             collate_fn=collate_fn)
    batch_x = iter(data_loader).next()
    print(batch_x)
    print('END')
    #以batch的最大长度为准，而不是整个数据的最大长度。

tensor([[[5.],
         [5.],
         [5.]],

        [[6.],
         [6.],
         [0.]]])
END


In [38]:
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data_length = [len(sq) for sq in data]
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data, data_length


if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=3, shuffle=True, 
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x, 
                                                  batch_x_len, batch_first=True)
print(batch_x,batch_x_len)
print(batch_x_pack)


tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [0.],
         [0.]],

        [[4.],
         [4.],
         [4.],
         [4.],
         [0.],
         [0.],
         [0.]]]) [7, 5, 4]
PackedSequence(data=tensor([[1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [1.],
        [1.]]), batch_sizes=tensor([3, 3, 3, 3, 2, 1, 1]))


In [39]:
x=batch_x_pack.data.reshape(-1,1)#the size -1 is inferred from other dimensions
print(x)
Linear=nn.Linear(1,1)#(input_dim,out_dim)
print(Linear(x))
#再将linear输出变回为pack 数据
len_s=batch_x_pack. batch_sizes
print(len_s)
h=[]
h.append(Linear(x).data[0:len_s[1]].reshape(1,-1))
h.append(Linear(x).data[(len_s[1]):(len_s[1]+len_s[2])].reshape(1,-1))
print(h)

tensor([[1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [4.],
        [1.],
        [3.],
        [1.],
        [1.]])
tensor([[-0.6299],
        [-2.4603],
        [-3.3756],
        [-0.6299],
        [-2.4603],
        [-3.3756],
        [-0.6299],
        [-2.4603],
        [-3.3756],
        [-0.6299],
        [-2.4603],
        [-3.3756],
        [-0.6299],
        [-2.4603],
        [-0.6299],
        [-0.6299]], grad_fn=<AddmmBackward>)
tensor([3, 3, 3, 3, 2, 1, 1])
[tensor([[-0.6299, -2.4603, -3.3756]]), tensor([[-0.6299, -2.4603, -3.3756]])]


In [40]:
if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=3, shuffle=True,
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x, batch_x_len, batch_first=True)

    net = nn.RNN(1, 10, 2,batch_first=True)

    print(batch_x_pack)
    h0 = torch.rand(2, 3, 10)
    out, (h1) = net(batch_x_pack,h0)
    print(out.data.shape)#第一位表示统计整个数据中共有多少个个非零的数据。即每个数据的RNN的输出。
    print(batch_x_pack.data.shape)
    print(out.batch_sizes)
    print(batch_x_pack.batch_sizes)
    print(out)
    print(h1.shape)#与h0的shape相同。
    out_pad, out_len = rnn_utils.pad_packed_sequence(out, batch_first=True)
    print(out_pad.shape)#第一位是句子数，第二位是时间步长（最长的句长），第三维是特征向量维度。
    print(out_len)#每一个句子的长度
    print(out_pad[1])#假如下标为1的句子真实长度为2，其余位（假如长度位3），共5位，那么out_pad[1]只有前两行有值，其余行为0（3行），
    #表示每一个特征映射为hidden_size，其余pad0.

PackedSequence(data=tensor([[3.],
        [5.],
        [6.],
        [3.],
        [5.],
        [6.],
        [3.],
        [5.],
        [3.],
        [3.]]), batch_sizes=tensor([3, 3, 2, 1, 1]))
torch.Size([10, 10])
torch.Size([10, 1])
tensor([3, 3, 2, 1, 1])
tensor([3, 3, 2, 1, 1])
PackedSequence(data=tensor([[ 0.7237, -0.7593,  0.4972,  0.3638,  0.0025,  0.1512, -0.3002, -0.1776,
          0.5667,  0.4305],
        [ 0.4755, -0.8211,  0.6660,  0.3290,  0.1670,  0.4700, -0.3020, -0.1033,
          0.6025,  0.6784],
        [ 0.5471, -0.8599,  0.6810,  0.1934,  0.4809,  0.3222, -0.7129, -0.0470,
          0.3924,  0.8368],
        [ 0.6286, -0.7171,  0.5843, -0.2735, -0.1872, -0.3548, -0.7369, -0.1785,
         -0.3591,  0.4875],
        [ 0.8204, -0.8250,  0.5631, -0.2583,  0.1999, -0.3318, -0.8290,  0.1107,
         -0.3554,  0.5253],
        [ 0.8422, -0.8556,  0.6322, -0.4389,  0.2007, -0.4008, -0.8284,  0.1744,
         -0.4705,  0.4718],
        [ 0.6102, -0.6494,  0.7335, -0

In [42]:
#完整
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader
import torch.utils.data as data


train_x = [torch.Tensor([[1], [1], [1], [1], [1], [1], [1]]),
           torch.Tensor([[2], [2], [2], [2], [2], [2]]),
           torch.Tensor([[3], [3], [3], [3], [3]]),
           torch.Tensor([[4], [4], [4], [4]]),
           torch.Tensor([[5], [5], [5]]),
           torch.Tensor([[6], [6]]),
           torch.Tensor([[7]])
           ]

class MyData(data.Dataset):
    def __init__(self, data_seq):
        self.data_seq = data_seq

    def __len__(self):
        return len(self.data_seq)

    def __getitem__(self, idx):
        return self.data_seq[idx]


def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data_length = [len(sq) for sq in data]
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data, data_length


if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=3, shuffle=True,
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x,
                                                  batch_x_len, batch_first=True)
    print(batch_x)
    print(batch_x_pack)
    net = nn.RNN(1, 10, 2, batch_first=True)
    h0 = torch.rand(2, 3, 10)
    #c0 = torch.rand(2, 3, 10)
    out, h1 = net(batch_x_pack, h0)
    out_pad, out_len = rnn_utils.pad_packed_sequence(out, batch_first=True)
    print(out)

tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [2.],
         [0.]],

        [[3.],
         [3.],
         [3.],
         [3.],
         [3.],
         [0.],
         [0.]]])
PackedSequence(data=tensor([[1.],
        [2.],
        [3.],
        [1.],
        [2.],
        [3.],
        [1.],
        [2.],
        [3.],
        [1.],
        [2.],
        [3.],
        [1.],
        [2.],
        [3.],
        [1.],
        [2.],
        [1.]]), batch_sizes=tensor([3, 3, 3, 3, 3, 2, 1]))
PackedSequence(data=tensor([[-0.6259,  0.1715,  0.5444,  0.8829,  0.1278,  0.1367,  0.1361,  0.2764,
         -0.3295, -0.4043],
        [-0.5708,  0.0593,  0.3468,  0.8746,  0.1221, -0.2615,  0.0860,  0.5150,
         -0.2896, -0.3114],
        [-0.3326,  0.1173,  0.3127,  0.7802,  0.1912,  0.0668,  0.0624,  0.2617,
         -0.2021, -0.3893],
        [-0.740